In [1]:
from xgboost import plot_importance
from xgboost import XGBClassifier
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from statsmodels.api import Logit
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

from sklearn.tree import export_graphviz
import graphviz
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import f1_score, roc_curve, auc
from sklearn.svm import SVC
from sklearn.metrics import f1_score, roc_curve, auc,roc_auc_score
from sklearn.neighbors import KNeighborsClassifier

import scipy.stats
from scipy import stats
%matplotlib inline
matplotlib.rc('font',family = 'Malgun Gothic')

In [2]:
# 전처리
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler

# 머신러닝 알고리즘
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import VotingClassifier

# 모델 튜닝 및 평가
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_predict
from sklearn import model_selection
import statsmodels.api as sm


In [3]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [4]:
import warnings
warnings.filterwarnings(action='ignore')

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler

In [6]:
df_raw = pd.read_csv("Dataset_all.csv")
df_raw.drop('Unnamed: 0',axis=1,inplace=True)

In [9]:
# 더미 변수화
df_raw[['Ox_Chamber']]=df_raw[['Ox_Chamber']].astype(str)
df_raw[['photo_soft_Chamber']]=df_raw[['photo_soft_Chamber']].astype(str)
df_raw[['lithography_Chamber']]=df_raw[['lithography_Chamber']].astype(str)
df_raw[['Etching_Chamber']]=df_raw[['Etching_Chamber']].astype(str)
df_raw[['Chamber_Num']]=df_raw[['Chamber_Num']].astype(str)
df_raw[['Wavelength']]=df_raw[['Wavelength']].astype(str)

# wet, dry 구분
# df_raw.loc[df_raw['type']=='dry','type']=0
# df_raw.loc[df_raw['type']=='wet','type']=1

# Vapor / H2O, O2 구분
df_raw.loc[df_raw['Vapor']=='H2O','Vapor']=0
df_raw.loc[df_raw['Vapor']=='O2','Vapor']=1

# Wavelength 365, 405, 436
df_raw.loc[df_raw['Wavelength']=='365','Wavelength']=0
df_raw.loc[df_raw['Wavelength']=='405','Wavelength']=1
df_raw.loc[df_raw['Wavelength']=='436','Wavelength']=2


# 필요 없는 변수 material, current, lamp, chamber, type제거
# df_list_before=['Temp_OXid', 'Vapor', 'ppm','Pressure', 'Oxid_time', 'thickness',
#         'resist_target', 'N2_HMDS', 'pressure_HMDS', 'temp_HMDS','temp_HMDS_bake', 'time_HMDS_bake', 'spin1', 'spin2', 'spin3','photoresist_bake', 'temp_softbake', 'time_softbake',
#         'Line_CD','Wavelength', 'Resolution', 'Energy_Exposure',
#         'Thin F4','Thin F2','Thin F3','Thin F1', 'Temp_Etching','Source_Power', 'Selectivity',
#         'Flux60s', 'Flux90s', 'Flux160s', 'Flux480s','Flux840s', 'input_Energy', 'Temp_implantation','Furance_Temp', 'RTA_Temp']

# df_dummy=pd.get_dummies(df_raw[df_list_before])

# # 불량판단 기준 달라질 것. 이 파라미터에 따른 성능 생각할 필요
# Y_wet.loc[Y_wet.Target<183]=0
# Y_wet.loc[Y_wet.Target>=183]=1
# Y_dry.loc[Y_dry.Target<183]=0
# Y_dry.loc[Y_dry.Target>=183]=1
# len(X),len(Y)
# df_train_x,df_test_x,df_train_y,df_test_y = train_test_split(X,Y,test_size = 0.3,random_state = 1234)

In [10]:
df_raw.loc[df_raw.Target>=183,'Quality']=1
df_raw.loc[df_raw.Target<183,'Quality']=0

In [11]:
# 제일 좋은 것 같은데
imp_features=['Temp_OXid', 'Thin F4', 'Thin F2', 'Flux480s', 'Source_Power',
        'thickness', 'Pressure', 'Energy_Exposure',
       'photoresist_bake', 'spin2', 'temp_HMDS_bake','Temp_implantation']
df_dummy=pd.get_dummies(df_raw[imp_features])
# ppm 제거

In [12]:
df_train_x,df_test_x,df_train_y,df_test_y = train_test_split(df_dummy,df_raw['Quality'],test_size = 0.3,random_state = 1234)

In [13]:
xgb_model = XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, gamma=0, gpu_id=-1,
                               importance_type='gain',
                               interaction_constraints='',
                               learning_rate=0.300000012, max_delta_step=0,
                               max_depth=5, max_leaves=1, min_child_weight=1,
                               monotone_constraints='()',
                               n_estimators=100, n_jobs=4, num_parallel_tree=1,
                               process_type='default', random_state=1234,
                               reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                               subsample=0.8, tree_method='exact',
                               validate_parameters=1, verbosity=None)

In [169]:
accuracy_test=[]
precision=[]
recall=[]
F1=[]

## XGB

In [170]:
xgb_model.fit(df_train_x,df_train_y)

[11:58:25] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=5,
              max_leaves=1, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=4,
              num_parallel_tree=1, objective='binary:logistic',
              process_type='default', random_state=1234, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=0.8,
              tree_method='exact', use_label_encoder=True,
              validate_parameters=1, verbosity=None)

In [171]:
y_pred = xgb_model.predict_proba(df_test_x)
threshold=0.2
y_pred[y_pred[:,1]>=threshold]=1
y_pred[y_pred[:,1]<threshold]=0
# print('Confustion matrix : \n{}'.format(confusion_matrix(df_test_y, y_pred[:,1])))
print('Confustion matrix : \n{}'.format(confusion_matrix(df_test_y, y_pred[:,1])))
print(accuracy_score(df_test_y, y_pred[:,1]))
print(precision_score(df_test_y, y_pred[:,1]))
print(recall_score(df_test_y, y_pred[:,1]))
print(f1_score(df_test_y, y_pred[:,1]))

Confustion matrix : 
[[217   7]
 [ 10  15]]
0.9317269076305221
0.6818181818181818
0.6
0.6382978723404256


In [172]:
accuracy_test.append(accuracy_score(df_test_y, y_pred[:,1]))
precision.append(precision_score(df_test_y, y_pred[:,1]))
recall.append(recall_score(df_test_y, y_pred[:,1]))
F1.append(f1_score(df_test_y, y_pred[:,1]))

## GB

In [173]:
gb_fianl=GradientBoostingClassifier(ccp_alpha=0.0,
                            criterion='friedman_mse', init=None,
                            learning_rate=0.1, loss='deviance',
                            max_depth=3, max_features=None,
                            max_leaf_nodes=None,
                            min_impurity_decrease=0.0,
                            min_impurity_split=None,
                            min_samples_leaf=1,
                            min_samples_split=2,
                            min_weight_fraction_leaf=0.0,
                            n_estimators=100,
                            n_iter_no_change=None,
                            presort='deprecated',
                            random_state=None, subsample=1.0,
                            tol=0.0001, validation_fraction=0.1,
                            verbose=0, warm_start=False)
gb_fianl.fit(df_train_x,df_train_y)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [174]:
y_pred = gb_fianl.predict_proba(df_test_x)
threshold=0.2
y_pred[y_pred[:,1]>=threshold]=1
y_pred[y_pred[:,1]<threshold]=0
print('Confustion matrix : \n{}'.format(confusion_matrix(df_test_y, y_pred[:,1])))
print(accuracy_score(df_test_y, y_pred[:,1]))
print(precision_score(df_test_y, y_pred[:,1]))
print(recall_score(df_test_y, y_pred[:,1]))
print(f1_score(df_test_y, y_pred[:,1]))

Confustion matrix : 
[[212  12]
 [  8  17]]
0.9196787148594378
0.5862068965517241
0.68
0.6296296296296295


In [175]:
accuracy_test.append(accuracy_score(df_test_y, y_pred[:,1]))
precision.append(precision_score(df_test_y, y_pred[:,1]))
recall.append(recall_score(df_test_y, y_pred[:,1]))
F1.append(f1_score(df_test_y, y_pred[:,1]))

## RF

In [176]:
rf_final=RandomForestClassifier(bootstrap=True,
                       ccp_alpha=0.0,
                       class_weight=None,
                       criterion='gini',
                       max_depth=None,
                       max_features='auto',
                       max_leaf_nodes=None,
                       max_samples=None,
                       min_weight_fraction_leaf=0.0,
                       n_estimators=100,
                       n_jobs=None,
                       oob_score=False,
                       random_state=None,
                       verbose=0,
                       warm_start=False)
rf_final.fit(df_train_x,df_train_y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [177]:
y_pred = rf_final.predict_proba(df_test_x)
threshold=0.2
y_pred[y_pred[:,1]>=threshold]=1
y_pred[y_pred[:,1]<threshold]=0
print('Confustion matrix : \n{}'.format(confusion_matrix(df_test_y, y_pred[:,1])))
print(accuracy_score(df_test_y, y_pred[:,1]))
print(precision_score(df_test_y, y_pred[:,1]))
print(recall_score(df_test_y, y_pred[:,1]))
print(f1_score(df_test_y, y_pred[:,1]))

Confustion matrix : 
[[202  22]
 [  3  22]]
0.8995983935742972
0.5
0.88
0.6376811594202899


In [178]:
accuracy_test.append(accuracy_score(df_test_y, y_pred[:,1]))
precision.append(precision_score(df_test_y, y_pred[:,1]))
recall.append(recall_score(df_test_y, y_pred[:,1]))
F1.append(f1_score(df_test_y, y_pred[:,1]))

## DT

In [179]:
dt_final=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                        criterion='gini', max_depth=4,
                                        max_features=None, max_leaf_nodes=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=20,
                                        min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        presort='deprecated', random_state=None,
                                        splitter='best')
dt_final.fit(df_train_x,df_train_y)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=4, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=20, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [180]:
y_pred = dt_final.predict_proba(df_test_x)
threshold=0.2
y_pred[y_pred[:,1]>=threshold]=1
y_pred[y_pred[:,1]<threshold]=0
print('Confustion matrix : \n{}'.format(confusion_matrix(df_test_y, y_pred[:,1])))
print(accuracy_score(df_test_y, y_pred[:,1]))
print(precision_score(df_test_y, y_pred[:,1]))
print(recall_score(df_test_y, y_pred[:,1]))
print(f1_score(df_test_y, y_pred[:,1]))

Confustion matrix : 
[[204  20]
 [  2  23]]
0.9116465863453815
0.5348837209302325
0.92
0.6764705882352942


In [181]:
accuracy_test.append(accuracy_score(df_test_y, y_pred[:,1]))
precision.append(precision_score(df_test_y, y_pred[:,1]))
recall.append(recall_score(df_test_y, y_pred[:,1]))
F1.append(f1_score(df_test_y, y_pred[:,1]))

## SVM

In [182]:
svm_final=SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', degree=3,
                     gamma='scale', kernel='rbf', max_iter=-1,
                     probability=True, random_state=None, shrinking=True,
                     tol=0.001, verbose=False)
svm_final.fit(df_train_x,df_train_y)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

In [183]:
y_pred = svm_final.predict_proba(df_test_x)
threshold=0.2
y_pred[y_pred[:,1]>=threshold]=1
y_pred[y_pred[:,1]<threshold]=0
print('Confustion matrix : \n{}'.format(confusion_matrix(df_test_y, y_pred[:,1])))
print(accuracy_score(df_test_y, y_pred[:,1]))
print(precision_score(df_test_y, y_pred[:,1]))
print(recall_score(df_test_y, y_pred[:,1]))
print(f1_score(df_test_y, y_pred[:,1]))

Confustion matrix : 
[[186  38]
 [ 24   1]]
0.751004016064257
0.02564102564102564
0.04
0.03125


In [184]:
accuracy_test.append(accuracy_score(df_test_y, y_pred[:,1]))
precision.append(precision_score(df_test_y, y_pred[:,1]))
recall.append(recall_score(df_test_y, y_pred[:,1]))
F1.append(f1_score(df_test_y, y_pred[:,1]))

## 로지스틱 회귀

In [185]:
df_raw_logit=df_raw
df_raw_logit.rename(columns = {'Thin F1' : 'Thin_F1','Thin F2' : 'Thin_F2','Thin F3' : 'Thin_F3','Thin F4' : 'Thin_F4'}, inplace = True)
df_train_logit, df_test_logit = train_test_split(df_raw_logit, test_size = 0.3, random_state = 1234)

In [186]:
# from_formula 함수를 이용하여 변수 역할 지정
# 분석 대상 이벤트 = “1”
log_model = Logit.from_formula("""Quality ~ Thin_F4 + Thin_F2 + Thin_F3 + Pressure + Flux90s +
 ppm""", df_train_logit)
# 적합
log_result = log_model.fit()
# 결과 출력
print(log_result.summary())

Optimization terminated successfully.
         Current function value: 0.151028
         Iterations 9
                           Logit Regression Results                           
Dep. Variable:                Quality   No. Observations:                  581
Model:                          Logit   Df Residuals:                      580
Method:                           MLE   Df Model:                            0
Date:                Thu, 18 Mar 2021   Pseudo R-squ.:                  0.5117
Time:                        11:59:10   Log-Likelihood:                -87.747
converged:                       True   LL-Null:                       -179.70
Covariance Type:            nonrobust   LLR p-value:                       nan
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   -265.0517     47.530     -5.577      0.000    -358.209    -171.894
Thin_F4        0.0172      0.

In [187]:
y_pred = log_result.predict(df_test_logit)
th = 0.2
# 0과 1의 값을 가진 class로 변환
y_pred_class = (y_pred > th).astype(int)
# 실제 test 데이터의 목표변수와 test 데이터의 예측값 비교
print("Accuracy: {0:.3f}".format(accuracy_score(df_test_logit["Quality"], y_pred_class)))
print("f1: {0:.3f}".format(f1_score(df_test_logit["Quality"], y_pred_class)))
print("recall: {0:.3f}\n".format(recall_score(df_test_logit["Quality"], y_pred_class)))

print("Confusion Matrix: \n{}".format(confusion_matrix(df_test_logit["Quality"], y_pred_class)))

Accuracy: 0.859
f1: 0.507
recall: 0.720

Confusion Matrix: 
[[196  28]
 [  7  18]]


In [188]:
accuracy_test.append(accuracy_score(df_test_logit["Quality"], y_pred_class))
precision.append(precision_score(df_test_logit["Quality"], y_pred_class))
recall.append(recall_score(df_test_logit["Quality"], y_pred_class))
F1.append(f1_score(df_test_logit["Quality"], y_pred_class))

In [189]:
accuracy_test

[0.9317269076305221,
 0.9196787148594378,
 0.8995983935742972,
 0.9116465863453815,
 0.751004016064257,
 0.8594377510040161]

In [190]:
precision

[0.6818181818181818,
 0.5862068965517241,
 0.5,
 0.5348837209302325,
 0.02564102564102564,
 0.391304347826087]

In [191]:
recall

[0.6, 0.68, 0.88, 0.92, 0.04, 0.72]

In [192]:
F1

[0.6382978723404256,
 0.6296296296296295,
 0.6376811594202899,
 0.6764705882352942,
 0.03125,
 0.5070422535211268]

In [193]:
final_model=pd.DataFrame({'accuracy':accuracy_test,'precision':precision,'recall':recall,'F1':F1},index=['XGB','GB','RF','DT','SVM','Logistic'])

In [194]:
final_model.T


,XGB,GB,RF,DT,SVM,Logistic
accuracy,0.931727,0.919679,0.899598,0.911647,0.751004,0.859438
precision,0.681818,0.586207,0.500000,0.534884,0.025641,0.391304
recall,0.600000,0.680000,0.880000,0.920000,0.040000,0.720000
F1,0.638298,0.629630,0.637681,0.676471,0.031250,0.507042
